In [1]:
import rioxarray as rxr
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import date,datetime
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
raster_data = rxr.open_rasterio('Curah_Hujan.tif')
mask = raster_data != -9999

# Calculate the minimum value excluding -9999
min_value_ch = raster_data.where(mask).min()
max_value_ch = raster_data.where(mask).max()

In [3]:
raster_data = rxr.open_rasterio('RH.tif')
mask = raster_data != -9999

# Calculate the minimum value excluding -9999
min_value_rh = raster_data.where(mask).min()
max_value_rh = raster_data.where(mask).max()

In [4]:
raster_data = rxr.open_rasterio('TEMPERATUR.tif')
mask = raster_data != -9999

# Calculate the minimum value excluding -9999
min_value_t = raster_data.where(mask).min()
max_value_t = raster_data.where(mask).max()

In [5]:
def mosaic_monthly(raster_file,year):
    raster_data = rxr.open_rasterio(raster_file)
    date_a = date(2011, 1, 1)
    ar_year=[]
    for i in range(1,13):
        date_b = date(year, i, 1)
        if i<11:
            date_c = date(year, i+1, 1)
        else:
            date_c = date(year+1,1,1)
        days_start=(date_b - date_a).days
        days_end=(date_c-date_a).days
        bands=raster_data[range(days_start,days_end+1)]
        ar_months = np.array([band.values for band in bands])
        ar_year.append(ar_months.mean(axis=0))        
    return np.array(ar_year)

In [8]:
dict_ch={}
for i in tqdm(range(2011,2023)):
    ch_mosaic=mosaic_monthly('Curah_Hujan.tif',i)
    dict_ch[i]=ch_mosaic

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [03:01<00:00, 15.15s/it]


In [9]:
dict_rh={}
for i in tqdm(range(2011,2023)):
    rh_mosaic=mosaic_monthly('RH.tif',i)
    dict_rh[i]=ch_mosaic

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:47<00:00,  8.96s/it]


In [10]:
dict_t={}
for i in tqdm(range(2011,2023)):
    t_mosaic=mosaic_monthly('TEMPERATUR.tif',i)
    dict_t[i]=t_mosaic

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [03:02<00:00, 15.18s/it]


In [11]:
tif_path = 'NDVI_11_24_full.tif'
raster = rxr.open_rasterio(tif_path)
# Convert the raster data into a NumPy array
ndvi_mosaic = raster.values

tif_path = 'PDSI_11_24_full.tif'
raster = rxr.open_rasterio(tif_path)
# Convert the raster data into a NumPy array
pdsi_mosaic = raster.values

print(ndvi_mosaic.shape)

(163, 38, 50)


In [12]:
##get ndvi from 2018-2020 for calibration
def months_between(start_date, end_date):
  """Calculates the number of months between two dates."""
  diff = relativedelta(end_date, start_date)
  return diff.years * 12 + diff.months

start_date = datetime(2011, 1, 1)
end_date_a = datetime(2020, 1, 1)
end_date_b = datetime(2023, 1, 1)
months_a = months_between(start_date, end_date_a)
months_b = months_between(start_date, end_date_b)

In [13]:
ndvi_cal=ndvi_mosaic[months_a:months_b,:,:]
pdsi_cal=pdsi_mosaic[months_a:months_b,:,:]

In [15]:
ch_cal=[]
rh_cal=[]
t_cal=[]
for i in [2020,2021,2022]:
    ch_cal.extend(dict_ch[i])
    rh_cal.extend(dict_rh[i])
    t_cal.extend(dict_t[i])
ch_cal=np.array(ch_cal)
rh_cal=np.array(rh_cal)
t_cal=np.array(t_cal)

In [21]:
dt_gab={}
dt_gab['ch']=ch_cal
dt_gab['rh']=rh_cal
dt_gab['t']=t_cal
dt_gab['ndvi']=ndvi_cal

In [23]:
import pickle
with open('data_gab_awal_2020_2022.pickle','wb') as file:
    pickle.dump(dt_gab,file)